In [1]:
# Installing libraries: 
# pip install geopy 

In [2]:
# 1
# removing duplciate lane entries
# removing lane info "_lane" from dataset
import pandas as pd

def clean_dataset(df):
    # Group the data by the two columns and keep only the first occurrence of each group
    reduced_df = df.drop_duplicates(subset=[5, 6], keep='first')
    # Convert column 1 values to strings and remove anything after and including the last underscore
    #reduced_df[1] = reduced_df[1].astype(str).str.rsplit('_', n=1).str[0]
    reduced_df.loc[:, 1] = reduced_df.loc[:, 1].astype(str).str.rsplit('_', n=1).str[0]

    reduced_df.iloc[:, 5] = reduced_df.iloc[:, 5].astype(str).apply(lambda x: float(x[:2] + '.' + x[2:]))
    reduced_df.iloc[:, 6] = reduced_df.iloc[:, 6].astype(str).apply(lambda x: float(x[:4] + '.' + x[4:]))

    return reduced_df


In [3]:
# 2
# group data based on sensor location description 
import pandas as pd

def get_sensor_group(data, sensor_loc, ignore=None):
    # Filter the DataFrame based on column 3 and ignore string
    mask = data[3].str.startswith(sensor_loc) & (~data[3].str.contains(ignore) if ignore is not None else True)
    matching_group = data[mask]
    return matching_group


In [ ]:
# Connecting the sensor groups based on travelling-salesman-problem algorithm:
# Its working as intended
# Make it a function to take sensor group and to produce a variable sensor adjacency list as well

import pandas as pd
import folium
from geopy.distance import distance

def calculate_distance(sensor_data):
    # Calculate distances between each pair of sensors
    num_sensors = len(sensor_data)
    distances = [[0.0] * num_sensors for _ in range(num_sensors)]
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            coord1 = (sensor_data.iloc[i][5], sensor_data.iloc[i][6])
            coord2 = (sensor_data.iloc[j][5], sensor_data.iloc[j][6])
            dist = distance(coord1, coord2).meters
            distances[i][j] = dist
            distances[j][i] = dist
    return distances

def tsp(sensor_data):
    
    distances = calculate_distance(sensor_data)
    map = folium.Map(location=[sensor_data[5].mean(), sensor_data[6].mean()], zoom_start=10)

    # Connect sensors with the shortest distance
    num_sensors = len(sensor_data)
    connected = set()
    min_distance = float('inf')
    min_i, min_j = -1, -1

    # Find the pair of sensors with the shortest distance
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            if distances[i][j] < min_distance:
                min_distance = distances[i][j]
                min_i, min_j = i, j

    # Connect the pair of sensors with a polyline
    if min_i != -1 and min_j != -1:
        sensor1 = sensor_data.iloc[min_i]
        sensor2 = sensor_data.iloc[min_j]
        folium.PolyLine([(sensor1[5], sensor1[6]), (sensor2[5], sensor2[6])],
                        color='red').add_to(map)
        connected.add(min_i)
        connected.add(min_j)

    # Connect the remaining sensors
    while len(connected) < num_sensors:
        min_distance = float('inf')
        min_i = -1
        for i in range(num_sensors):
            if i not in connected:
                for j in connected:
                    if distances[i][j] < min_distance:
                        min_distance = distances[i][j]
                        min_i = i
                        min_j = j
        if min_i != -1:
            sensor1 = sensor_data.iloc[min_i]
            sensor2 = sensor_data.iloc[min_j]
            folium.PolyLine([(sensor1[5], sensor1[6]), (sensor2[5], sensor2[6])],
                            color='red').add_to(map)
            connected.add(min_i)

    
    for _, sensor in sensor_data.iterrows():
        tooltip_text = f"Sensor_ID: {sensor[1]}, Des: {sensor[3]}"
        folium.CircleMarker(
            location=[sensor[5], sensor[6]],
            radius=5,
            color='blue',
            fill=True,
            fill_color='blue',
            tooltip=tooltip_text  # Add tooltip
            ).add_to(map)
    return map



In [ ]:
# Print adj list using the list of sensors: 

import pandas as pd
import folium
from geopy.distance import distance

def calculate_distance(sensor_data):
    # Calculate distances between each pair of sensors
    num_sensors = len(sensor_data)
    distances = [[0.0] * num_sensors for _ in range(num_sensors)]
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            coord1 = (sensor_data.iloc[i][5], sensor_data.iloc[i][6])
            coord2 = (sensor_data.iloc[j][5], sensor_data.iloc[j][6])
            dist = distance(coord1, coord2).meters
            distances[i][j] = dist
            distances[j][i] = dist
    return distances

def tsp(sensor_data):
    distances = calculate_distance(sensor_data)
    
    # Create an empty list to store the adjacency list
    adjacency_list = []

    # Connect sensors with the shortest distance
    num_sensors = len(sensor_data)
    connected = set()
    min_distance = float('inf')
    min_i, min_j = -1, -1

    # Find the pair of sensors with the shortest distance
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            if distances[i][j] < min_distance:
                min_distance = distances[i][j]
                min_i, min_j = i, j

    # Connect the pair of sensors
    if min_i != -1 and min_j != -1:
        adjacency_list.append([sensor_data.iloc[min_i][1], sensor_data.iloc[min_j][1], min_distance])
        connected.add(min_i)
        connected.add(min_j)

    # Connect the remaining sensors
    while len(connected) < num_sensors:
        min_distance = float('inf')
        min_i = -1
        for i in range(num_sensors):
            if i not in connected:
                for j in connected:
                    if distances[i][j] < min_distance:
                        min_distance = distances[i][j]
                        min_i = i
                        min_j = j
        if min_i != -1:
            adjacency_list.append([sensor_data.iloc[min_i][1], sensor_data.iloc[min_j][1], min_distance])
            connected.add(min_i)

    # Create a DataFrame from the adjacency list
    adjacency_df = pd.DataFrame(adjacency_list, columns=['sensor1', 'sensor2', 'cost'])
    return adjacency_df




In [6]:
# Run the full process

# def main():

# Read from the original data and claen it:
data = clean_dataset(pd.read_csv('../detector/detectors2018.csv', header=None))

# Select sensor group:

#sensor_loc = "CC-215 WB"
#sensor_loc="CC-215 EB"
#sensor_loc="I-15 NB"
#sensor_loc= "I-515 SB"
#sensor_loc="US-95 SB"
sensor_loc="US-95 NB"


# Ignore "-" from the sensor group
ignore = 'Ramp'

# Get list of sensors using the sensor group
sensor_group = get_sensor_group(data, sensor_loc, ignore)

# Plot the map of the sensor group 

#sensor_map = tsp(sensor_group)
#sensor_map


# Call the tsp function to get the sensor adjacency list
adjacency_list = tsp(sensor_group)

# Print the adjacency list
print(adjacency_list)

C:\Users\MSI\AppData\Local\Temp\ipykernel_21296\1195696214.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df.loc[:, 1] = reduced_df.loc[:, 1].astype(str).str.rsplit('_', n=1).str[0]
C:\Users\MSI\AppData\Local\Temp\ipykernel_21296\1195696214.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df.iloc[:, 5] = reduced_df.iloc[:, 5].astype(str).apply(lambda x: float(x[:2] + '.' + x[2:]))
C:\Users\MSI\AppData\Local\Temp\ipykernel_21296\1195696214.py:14: SettingWithCopyWarning: 
A value is 

      sensor1    sensor2        cost
0   421_3_507  452_1_508  398.821527
1   421_2_506  421_3_507  521.119646
2   421_1_505  421_2_506  543.986851
3   420_3_160  421_1_505  529.107786
4   420_2_160  420_3_160  519.764609
5   420_1_231  420_2_160  559.560881
6   419_3_217  420_1_231  575.378420
7   419_2_217  419_3_217  515.204372
8   419_1_234  419_2_217  537.507121
9   418_3_234  419_1_234  511.763187
10  418_2_225  418_3_234  490.235917
11  418_1_225  418_2_225  565.468593
12  417_3_230  418_1_225  563.803664
13  417_2_230  417_3_230  539.065230
14  417_1_216  417_2_230  541.163579
15  416_3_216  417_1_216  491.605855
16  416_2_224  416_3_216  508.964586
17  416_1_215  416_2_224  581.555789
18  415_3_215  416_1_215  582.886828
19  415_2_214  415_3_215  582.886316
20  415_1_223  415_2_214  522.311204
21  414_3_223  415_1_223  417.316073
22  414_2_209  414_3_223  418.640613
23  414_1_209  414_2_209  481.385717
24  413_3_204  414_1_209  522.964043
25  413_2_208  413_3_204  534.029685
2

In [ ]:
# Dataset Preparation based on sensor-group:
# Scripts takes in the sensor list from sensor group and merges the data files to a dataset

# Matching sensor data with sensor files downloaded:

# Read from the original data and claen it:
data = clean_dataset(pd.read_csv('../detector/detectors2018.csv', header=None))
# Get the list of sensors using the sensor group name:
sensor_loc="I-15 NB"
ignore = 'Ramp'
sensor_group = get_sensor_group(data, sensor_loc, ignore)

##########################################################################################

# This script merges individual sensor data files from a set of specific list of sensor name
import pandas as pd
import os
from tqdm import tqdm

# read the list of file names from the csv file
#file_list = pd.read_csv('/home/tzahid/Desktop/pred_sensor_list.csv')['pred_sensor'].tolist()
file_list=sensor_group[1]

# add '.csv' extension to file names in the list
file_list = ['d_' + filename + '.csv' for filename in file_list]

# get a list of all csv files in the folder
path = '/home/tzahid/Desktop/new2018dump/'
all_files = os.listdir(path)
csv_files = [filename for filename in all_files if filename.endswith('.csv') and filename in file_list]


print(f"{len(csv_files)} CSV files found matching with the list of file names.")


# get the range of index values
max_index = 0
for file in csv_files:
    try:
        data = pd.read_csv(os.path.join(path, file), usecols=[0])
        max_index = max(max_index, data['index_date'].max())
    except FileNotFoundError:
        print(f"File not found: {file}. Skipping to next file.")

index_range = range(1, max_index+1)

# create an empty dataframe with the desired index
df = pd.DataFrame(index=index_range)

# loop through the csv files and extract the 5th column and add it to the dataframe
for file in tqdm(csv_files, desc='Processing CSV files'):
    try:
        col_name = os.path.splitext(file)[0]
        col_name = col_name.replace(" ", "_")  # replace spaces with underscores in column names
        data = pd.read_csv(os.path.join(path, file), usecols=[0, 4], header=0, skiprows=[0], names=['index_date', col_name]) # modify usecols to define which columns to extract, here 0 and 4
        data = data.drop_duplicates(subset=['index_date'])  # drop any duplicate rows
        data = data.set_index('index_date')  # set the index to the 'index_date' column
        data = data.reindex(index_range, fill_value=0)  # add missing index rows and fill with 0
        df[col_name] = data[col_name]  # add the column to the main dataframe
    except FileNotFoundError:
        print(f"File not found: {file}. Skipping to next file.")

# save the concatenated dataframe to a new csv file
# name of the final file
_filename= sensor_loc +' without '+ ignore +'.csv'
df.to_csv(os.path.join(path, _filename), index_label='index_date')

# print the range of index values
print(f"Index range: 1 - {max_index}")

# print final message
print(f"Concatenation complete. {len(df.columns)} files have been concatenated.")



In [ ]:
import pandas as pd
import numpy as np
import miceforest as mf

In [ ]:
# Dataset Analysis

raw_data=pd.read_csv("E:/I-15 Datataset/2018/I-15 NB without Ramp.csv")
import matplotlib.pyplot as plt
import numpy as np

# Get the row and column number of the dataframe

num_rows, num_columns = raw_data.shape
print(f"raw data rows: {num_rows} columns: {num_columns}")
#raw_data.head(10)

# Replace 0 by NaN
data_masked = raw_data.mask(raw_data == 0, np.nan)
data_masked.head(15)

# Remove columns which has less than threshold amount of rows
threshold_percentage = 80
thresh_value = int((threshold_percentage / 100) * len(data_masked))
df_cleaned = data_masked.dropna(axis=1, thresh=thresh_value)
num_rows, num_columns = df_cleaned.shape
print(f"cleaned data rows: {num_rows} columns: {num_columns}")

# Display the NaN percentage for each column and total
nan_percentage = df_cleaned.isna().mean() * 100
total_nan_percentage=nan_percentage.mean()

print(f"Column NaN %: \n{nan_percentage}")
print(f"Total NaN %: {total_nan_percentage}")

#print(df_cleaned)
#df_cleaned.head(30)

# Impute the dataframe with NaN values

kds = mf.ImputationKernel(
  df_cleaned,
  datasets=1,
  save_all_iterations=True,
  random_state=1991
)

# Run the MICE algorithm for 3 iterations
kds.mice(3)

print("Data imputation complete!")
# Sample into 5/10/15 mins windows evenly

In [ ]:
data_complete= kds.complete_data()
data_complete

In [ ]:
data_complete= kds.complete_data()
data_complete.iloc[18684,:]

data_complete.columns 


In [ ]:
data_complete= kds.complete_data()
data_complete.head(10)

date_index=pd.read_csv("E:/I-15 Datataset/2018/dates.csv")
timed_data_complete= pd.merge(data_complete, date_index, left_on='index_date', right_on='date_index', how='inner')
# timed_data_complete


In [ ]:
import pandas as pd

timed_data_complete['date_time'] = pd.to_datetime(timed_data_complete['date_time'])
timed_data_complete.set_index('date_time', inplace=True)

timed_sampled = timed_data_complete.resample('5T').mean()

print(timed_sampled)

output_file_path = 'E:/I-15 Datataset/2018/5min_sampled_data.csv'  # Replace with the desired file path
timed_sampled.to_csv(output_file_path)


In [ ]:
T=date_index=pd.read_csv("E:/I-15 Datataset/2018/dates.csv")
T = pd.DataFrame(date_index, index=pd.to_datetime(date_index[date_time]))


# Check the inferred frequency
frequency = pd.infer_freq(df.index)
T